# **웹 로그 기반 조회수 예측 해커톤**

## 데이터셋 정보

Train Data
* Total number of rows: 252289
* Total number of columns: 19

Test Data
* Total number of rows: 79786
* Total number of columns:18

train에서 컬럼의 unique한 값의 개수가 20개 이하인 컬럼 : destktop, mobile device, tablet, new, bounced, transaction, continent, traffic_medium

## 전처리

- object type의 column 들은 Targetencoder를 사용하여 encoding.
- 결측값이 존재하는 컬럼들을 제거하였고, sessionID를 제거.
- 학습 데이터에서 중복 데이터를 제거.
- ChatGPT에게 질문하여, 새로운 파생변수를 생성.

## 모델링

- CatBoostRegressor을 사용했고, optuna를 이용한 hyperparameter 설정.
- optuna를 이용해 hyperparameter를 먼저 설정한 후 학습을 진행하였습니다.
- optuna 과정과 제출을 위한 학습 과정 모두 동일한 cross-validation 학습 방법을 사용.
- cross-validation은 StratifiedKFold를 사용하였으며, 이는 252,289개의 학습 데이터 중 Target의 nunique가 117개 밖에 존재하지 않아 사용할 수 있겠다고 판단.
- StratifiedKFold의 groups 변수에 학습 데이터의 'userID'를 할당하였으며, 데이터셋 분할 후 해당 컬럼을 제거.(이 방법을 사용했을 때, validation rmse가 크게 감소.)
- EDA를 통해 bounced가 1일 때는 항상 Target이 1인 것을 관찰하였고, 학습 과정에서 bounced가 0인 데이터만 학습.
- 각 fold마다 학습을 진행하였고, 학습된 모델을 테스트 데이터셋에 대해 추론을 하여 결과를 평균을 내는 Soft voting.
- EDA를 통해 Target의 범위가 1이상 386이하인 것을 관찰하였고, 모델의 예측 결과가 0.xx부터 92.xx 사이인 것을 확인하고, 1 이하로 예측된 결과값들을 1로 변환.
- bounced가 1인 데이터들의 예측값을 1로 변환.

## 차별점 및 배울점

bounced=1 처리한 것이 차별점이다. EDA를 통해 bounced가 1이면 항상 Target이 1임을 관찰하고, 학습에서 제외하고 후처리로 처리한 점.
이는 모델이 불필요하게 “확률적 예측”을 하지 않고, 도메인 지식을 적극 활용하여 결과를 보정했다. 이는 단순한 ML 접근이 아니라 EDA 기반 규칙 결합이라는 점에서 차별화된 것이다.

bounced=1 처리나 Target 최소값 조정처럼, 모델 + 규칙(rule-based) 결합 전략은 Kaggle 등 경쟁 환경에서 자주 쓰이는 유용한 기법임을 배울 수 있었다.
